In [7]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.2 MB/s eta 0:00:00


## Step 1: Imports & Configuration

In [8]:
import os
import cv2
import shutil
from tqdm import tqdm
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 2: Setting Directories for YOLO Model

In [3]:

# Original Dataset Path
train_dir = "/content/drive/MyDrive/Labmentix Projects/11.Aerial Object Classification and Detection Project/train"
val_dir = "/content/drive/MyDrive/Labmentix Projects/11.Aerial Object Classification and Detection Project/valid"
test_dir = "/content/drive/MyDrive/Labmentix Projects/11.Aerial Object Classification and Detection Project/test"

# YOLO dataset folder
output_dir = "/content/detection_dataset"
os.makedirs(output_dir, exist_ok=True)

splits = {
    "train": train_dir,
    "val": val_dir,
    "test": test_dir
}

classes = {"bird": 0, "drone": 1}

for split, input_path in splits.items():

    img_out = os.path.join(output_dir, "images", split)
    lbl_out = os.path.join(output_dir, "labels", split)

    os.makedirs(img_out, exist_ok=True)
    os.makedirs(lbl_out, exist_ok=True)

    for cls_name, cls_id in classes.items():
        folder = os.path.join(input_path, cls_name)
        images = os.listdir(folder)

        print(f"Processing {split}/{cls_name}...")

        for img_name in tqdm(images):
            img_path = os.path.join(folder, img_name)

            # Loading images
            img = cv2.imread(img_path)
            if img is None:
                continue

            h, w, _ = img.shape

            # Auto-Bounding Box Generation
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if len(contours) == 0:
                # fallback: full image bounding box
                x, y, bw, bh = 0, 0, w, h
            else:
                cnt = max(contours, key=cv2.contourArea)
                x, y, bw, bh = cv2.boundingRect(cnt)

            # Converting to YOLO format
            x_center = (x + bw / 2) / w
            y_center = (y + bh / 2) / h
            width = bw / w
            height = bh / h

            # Save image
            new_img_path = os.path.join(img_out, img_name)
            shutil.copy(img_path, new_img_path)

            # Save label
            label_name = img_name.replace(".jpg", ".txt").replace(".png", ".txt")
            label_path = os.path.join(lbl_out, label_name)

            with open(label_path, "w") as f:
                f.write(f"{cls_id} {x_center} {y_center} {width} {height}\n")


Processing train/bird...


100%|██████████| 1414/1414 [00:45<00:00, 31.33it/s]


Processing train/drone...


100%|██████████| 1248/1248 [00:32<00:00, 38.31it/s]


Processing val/bird...


...

100%|██████████| 225/225 [00:05<00:00, 44.27it/s]


Processing test/bird...


100%|██████████| 121/121 [00:03<00:00, 37.37it/s]


Processing test/drone...


100%|██████████| 94/94 [00:46<00:00,  2.04it/s]


In [4]:
#Checking if the path are right to give in data.yaml
for folder in [
    "/content/detection_dataset/images/train",
    "/content/detection_dataset/images/val",
    "/content/detection_dataset/images/test",
    "/content/detection_dataset/labels/train",
    "/content/detection_dataset/labels/val",
    "/content/detection_dataset/labels/test",
]:
    print(folder, "->", len(os.listdir(folder)))


/content/detection_dataset/images/train -> 2662
/content/detection_dataset/images/val -> 442
/content/detection_dataset/images/test -> 215
/content/detection_dataset/labels/train -> 2662
/content/detection_dataset/labels/val -> 442
/content/detection_dataset/labels/test -> 215


## Step 3: Creating data.yaml File

In [5]:
data_yaml = """
train: /content/detection_dataset/images/train
val: /content/detection_dataset/images/val
test: /content/detection_dataset/images/test

nc: 2
names: ['bird', 'drone']
"""

with open("/content/detection_dataset/data.yaml", "w") as f:
    f.write(data_yaml)

import os
print(os.path.exists("/content/detection_dataset/data.yaml"))  # should be True


True


## Step 4: Training YOLO Model

In [10]:
model = YOLO("yolov8s.pt")
model.train(
    data="/content/detection_dataset/data.yaml",
    epochs=50,
    imgsz=416,
    batch=8,
    verbose=True
)


Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/detection_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x786fc400e390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

- Precision = 0.89
- Recall = 0.76
- mAP50 = 0.88
- mAP50-95 = 0.76
- Fitness=0.76

##### My Assumptions
- Strong model for 2-class detection
- Precision and recall are well-balanced, so it’s neither missing objects nor producing too many false positives.


In [12]:
model = YOLO("/content/runs/detect/train2/weights/best.pt")
results = model.predict(
    source="/content/detection_dataset/images/test/",
    conf=0.25,
    save=True,
    show=True
)


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


image 1/215 /content/detection_dataset/images/test/00083b384685315d_jpg.rf.abfd1b2cc8c681777bae66d5327bb9ea.jpg: 416x416 1 bird, 10.2ms
image 2/215 /content/detection_dataset/images/test/00188d7f40a84793_jpg.rf.7f9da2b662dc236fbdcc1f22d8e0983e.jpg: 416x416 1 bird, 10.3ms
image 3/215 /content/detection_dataset/images/test/0028adf0e92c3da2_jpg.rf.48c97d30547a46e83abcbd1bf801c72f.jpg: 416x416 2 birds, 10.2ms
image 4/215 /content/detection_dataset/images/test/00347cbbeed4bedb_jpg.rf.aa19c9a7a22d76f8539ca12ac7cb82d5.jpg: 416x416 1 bird, 10.2ms
image 5/215 /content/detection_dataset/images/test/004cc8de466969bc_jpg.rf.ca1ffa31f30a92a3987e689583983973.jpg: 416x416 1 bird, 10.2ms
image 6/215 /content/detection_dataset/images/test/006ce62a72df4dd4_jpg.rf.8f8e7b6e9a26a01be91ea1b7dc4011d2.jpg: 416x416 1 bird, 10.2ms
image 7/215 /content/detection_dataset/images/test/006f6769070fc876_jpg.rf.bda2b07a999cd589936497151e0d009f.

In [13]:
# Zipping the entire runs folder ro download
shutil.make_archive("/content/train6_runs", 'zip', "/content/runs")


'/content/train6_runs.zip'